In [1]:
import pandas as pd 
import numpy as np
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error
import time
from sklearn.preprocessing import StandardScaler

In [2]:
Res=pd.read_csv("C:\programming_course\.vscode\major_project\Restaurant reviews.csv")

In [3]:
Res.head(10)

,Restaurant,Reviewer,Review,Rating,Metadata,Time,Pictures,7514
0,Beyond Flavours,Rusha Chakraborty,"The ambience was good, food was quite good . h...",5,"1 Review , 2 Followers",5/25/2019 15:54,0,2447.0
1,Beyond Flavours,Anusha Tirumalaneedi,Ambience is too good for a pleasant evening. S...,5,"3 Reviews , 2 Followers",5/25/2019 14:20,0,NaN
2,Beyond Flavours,Ashok Shekhawat,A must try.. great food great ambience. Thnx f...,5,"2 Reviews , 3 Followers",5/24/2019 22:54,0,NaN
3,Beyond Flavours,Swapnil Sarkar,Soumen das and Arun was a great guy. Only beca...,5,"1 Review , 1 Follower",5/24/2019 22:11,0,NaN
4,Beyond Flavours,Dileep,Food is good.we ordered Kodi drumsticks and ba...,5,"3 Reviews , 2 Followers",5/24/2019 21:37,0,NaN
5,Beyond Flavours,Nagabhavani K,"Ambiance is good, service is good, food is aPr...",5,1 Review,5/24/2019 15:22,0,NaN
6,Beyond Flavours,Jamuna Bhuwalka,"Its a very nice place, ambience is different, ...",5,1 Review,5/24/2019 1:02,0,NaN
7,Beyond Flavours,Sandhya S,Well after reading so many reviews finally vis...,4,1 Review,5/23/2019 15:01,0,NaN
8,Beyond Flavours,Akash Thorat,"Excellent food , specially if you like spicy f...",5,"1 Review , 1 Follower",5/22/2019 23:12,0,NaN
9,Beyond Flavours,Smarak Patnaik,Came for the birthday treat of a close friend....,5,"1 Review , 1 Follower",5/22/2019 22:37,0,NaN


In [4]:
Res=Res.drop(['Pictures','7514'],axis=1)

In [5]:
n_users = Res['Reviewer'].nunique()
n_restaurants = Res['Restaurant'].nunique()

print('Number of Unique Users: ', n_users)
print('Number of Restaurant: ',n_restaurants)

Number of Unique Users:  7446
Number of Restaurant:  100


In [6]:
Res.head(10)

,Restaurant,Reviewer,Review,Rating,Metadata,Time
0,Beyond Flavours,Rusha Chakraborty,"The ambience was good, food was quite good . h...",5,"1 Review , 2 Followers",5/25/2019 15:54
1,Beyond Flavours,Anusha Tirumalaneedi,Ambience is too good for a pleasant evening. S...,5,"3 Reviews , 2 Followers",5/25/2019 14:20
2,Beyond Flavours,Ashok Shekhawat,A must try.. great food great ambience. Thnx f...,5,"2 Reviews , 3 Followers",5/24/2019 22:54
3,Beyond Flavours,Swapnil Sarkar,Soumen das and Arun was a great guy. Only beca...,5,"1 Review , 1 Follower",5/24/2019 22:11
4,Beyond Flavours,Dileep,Food is good.we ordered Kodi drumsticks and ba...,5,"3 Reviews , 2 Followers",5/24/2019 21:37
5,Beyond Flavours,Nagabhavani K,"Ambiance is good, service is good, food is aPr...",5,1 Review,5/24/2019 15:22
6,Beyond Flavours,Jamuna Bhuwalka,"Its a very nice place, ambience is different, ...",5,1 Review,5/24/2019 1:02
7,Beyond Flavours,Sandhya S,Well after reading so many reviews finally vis...,4,1 Review,5/23/2019 15:01
8,Beyond Flavours,Akash Thorat,"Excellent food , specially if you like spicy f...",5,"1 Review , 1 Follower",5/22/2019 23:12
9,Beyond Flavours,Smarak Patnaik,Came for the birthday treat of a close friend....,5,"1 Review , 1 Follower",5/22/2019 22:37


In [7]:
Res.describe()

,Restaurant,Reviewer,Review,Rating,Metadata,Time
count,10000,9962,9955,9962,9962,9962
unique,100,7446,9364,10,2477,9782
top,Beyond Flavours,Parijat Ray,good,5,1 Review,7/29/2018 20:34
freq,100,13,237,3832,919,3


In [8]:
Res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Restaurant  10000 non-null  object
 1   Reviewer    9962 non-null   object
 2   Review      9955 non-null   object
 3   Rating      9962 non-null   object
 4   Metadata    9962 non-null   object
 5   Time        9962 non-null   object
dtypes: object(6)
memory usage: 468.9+ KB


In [9]:
Res.duplicated().sum()
Res.drop_duplicates(inplace=True)

#Remove the NaN values from the dataset
Res.isnull().sum()
Res.dropna(how='any',inplace=True)

Res['Rating'] = pd.to_numeric(Res['Rating'],errors = 'coerce')


In [10]:
res = list(Res['Restaurant'].unique())
Res['Mean Rating'] = 0

for i in range(len(res)):
    Res['Mean Rating'][Res['Restaurant'] == res[i]] = Res['Rating'][Res['Restaurant'] == res[i]].mean()

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (1,5))
Res[['Mean Rating']] = scaler.fit_transform(Res[['Mean Rating']]).round(2)

C:\Users\Ankita Singh\AppData\Local\Temp\ipykernel_12928\3778508026.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Res['Mean Rating'][Res['Restaurant'] == res[i]] = Res['Rating'][Res['Restaurant'] == res[i]].mean()


In [11]:
Res.head(10)

,Restaurant,Reviewer,Review,Rating,Metadata,Time,Mean Rating
0,Beyond Flavours,Rusha Chakraborty,"The ambience was good, food was quite good . h...",5.0,"1 Review , 2 Followers",5/25/2019 15:54,4.03
1,Beyond Flavours,Anusha Tirumalaneedi,Ambience is too good for a pleasant evening. S...,5.0,"3 Reviews , 2 Followers",5/25/2019 14:20,4.03
2,Beyond Flavours,Ashok Shekhawat,A must try.. great food great ambience. Thnx f...,5.0,"2 Reviews , 3 Followers",5/24/2019 22:54,4.03
3,Beyond Flavours,Swapnil Sarkar,Soumen das and Arun was a great guy. Only beca...,5.0,"1 Review , 1 Follower",5/24/2019 22:11,4.03
4,Beyond Flavours,Dileep,Food is good.we ordered Kodi drumsticks and ba...,5.0,"3 Reviews , 2 Followers",5/24/2019 21:37,4.03
5,Beyond Flavours,Nagabhavani K,"Ambiance is good, service is good, food is aPr...",5.0,1 Review,5/24/2019 15:22,4.03
6,Beyond Flavours,Jamuna Bhuwalka,"Its a very nice place, ambience is different, ...",5.0,1 Review,5/24/2019 1:02,4.03
7,Beyond Flavours,Sandhya S,Well after reading so many reviews finally vis...,4.0,1 Review,5/23/2019 15:01,4.03
8,Beyond Flavours,Akash Thorat,"Excellent food , specially if you like spicy f...",5.0,"1 Review , 1 Follower",5/22/2019 23:12,4.03
9,Beyond Flavours,Smarak Patnaik,Came for the birthday treat of a close friend....,5.0,"1 Review , 1 Follower",5/22/2019 22:37,4.03


In [12]:
Res["Review"] = Res["Review"].str.lower()

## Removal of Puctuations
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

Res["Review"] = Res["Review"].apply(lambda text: remove_punctuation(text))

## Removal of Stopwords
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

Res["Review"] = Res["Review"].apply(lambda text: remove_stopwords(text))


In [13]:
Res[['Review', 'Reviewer']].sample(5)

,Review,Reviewer
2620,worst place hang pathetic customer service pla...,Nithinesh Bodhanapu
1584,nyc,Deepali Goel
8056,second time oouder tandoori food works first t...,Deep Thakur
3908,cozy comfortable good place enjoy cup coffee c...,Pallavi K
1747,really disappointed food qualitythe food makes...,Debdut Modak


In [14]:
restaurant_names = list(Res['Restaurant'].unique())
def get_top_words(column, top_nu_of_words, nu_of_word):
    vec = CountVectorizer(ngram_range= nu_of_word, stop_words='english')
    bag_of_words = vec.fit_transform(column)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:top_nu_of_words]



In [15]:
df_percent = Res.sample(frac=0.5)

In [16]:
df_percent.set_index("Restaurant", inplace=True)
indices = pd.Series(df_percent.index)

# Creating tf-idf matrix
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0.0, stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_percent['Review'])

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [18]:
from sklearn.metrics.pairwise import pairwise_distances

In [26]:

def recommend(name, cosine_similarities = cosine_similarities):
    start_time=time.time()
    
    recommend_restaurant = []
    
    
    idx = indices[indices == name].index[0]
    
    
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)
    
    # Extract top 30 restaurant indexes with a similar cosine-sim value
    top30_indexes = list(score_series.iloc[0:31].index)
    
    # Names of the top 30 restaurants
    for each in top30_indexes:
        recommend_restaurant.append(list(df_percent.index)[each])
    
    # Creating the new data set to show similar restaurants
    df_new = pd.DataFrame(columns=[ 'Mean Rating','Metadata'])
    # Create the top 30 similar restaurants with some of their columns
    for each in recommend_restaurant:
        df_new = df_new._append(pd.DataFrame(df_percent[['Mean Rating', 'Metadata']][df_percent.index == each].sample()))
    
    # Drop the same named restaurants and sort only the top 10 by the highest rating
    df_new = df_new.drop_duplicates(subset=['Mean Rating', 'Metadata'], keep=False)
    df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(10)

    


    
    
    print('TOP %s RESTAURANTS LIKE %s WITH SIMILAR REVIEWS: ' % (str(len(df_new)), name))
    end_time=time.time()
    print("Execution time is :",end_time-start_time)
    
    
    return df_new

recommend('The Glass Onion')



TOP 10 RESTAURANTS LIKE The Glass Onion WITH SIMILAR REVIEWS: 
Execution time is : 0.14422821998596191


,Mean Rating,Metadata
Over The Moon Brew Company,4.13,"2 Reviews , 1 Follower"
Over The Moon Brew Company,4.13,"3 Reviews , 17 Followers"
Over The Moon Brew Company,4.13,"9 Reviews , 9 Followers"
Mazzo - Marriott Executive Apartments,3.89,"3 Reviews , 2 Followers"
Arena Eleven,3.77,"3 Reviews , 2 Followers"
NorFest - The Dhaba,3.63,"1 Review , 1 Follower"
American Wild Wings,3.54,"19 Reviews , 31 Followers"
Amul,3.47,"6 Reviews , 1 Follower"
PourHouse7,3.35,"56 Reviews , 156 Followers"
The Tilt Bar Republic,3.33,"2 Reviews , 1 Follower"
